# Test Transquest

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# NER : 

In [ ]:
import spacy
from spacy.tokens import Span


In [ ]:
!python -m spacy download en_core_web_trf
!python -m spacy download de_core_news_lg

In [ ]:
!pip install spacy-transformers

In [ ]:
import spacy_transformers
nlp = spacy.load('en_core_web_trf')
nlp_ger = spacy.load('de_core_news_lg') 

In [ ]:
list_ent = {
    "PRODUCT" : "product",
    "LOC" : "Himalayas",
    "DATE" : "this year",
    "TIME" : "night",
    "MONEY" : "three dollars",
    "PERSON" : "David",
    "ORG" : "IBM",
    "GPE" : "Paris",
    "PERCENT" : "four percent",
    "CARDINAL" : "three"
}

In [ ]:
def edit_ner(sen, nlp, list_ent):
    doc = nlp(sen)
    new_sentence = ""
    index_ent = 0 
    index_tok = 0
    while (index_tok < len(doc)):
        token = doc[index_tok]
        if token.ent_type_ != '':
            l_ent = -(doc.ents[index_ent].start-doc.ents[index_ent].end)
            if token.ent_type_ in list_ent:
                replacement_word = list_ent[token.ent_type_]
                new_sentence += replacement_word 
            else : 
                new_sentence += doc.ents[index_ent].text 
            index_tok+=l_ent
            index_ent+=1 
        else:
            new_sentence += token.text 
            index_tok+=1

        if (index_tok <len(doc) and not doc[index_tok].is_punct):
            new_sentence += token.whitespace_
    return str(new_sentence)

In [ ]:
list_ent_german = {
"PRODUCT" : "Produkt",
"LOC" : "Himalaya",
"DATE" : "dieses Jahr",
"TIME" : "Nacht",
"MONEY" : "drei Dollar",
"PERSON" : "David",
"ORG" : "IBM",
"GPE" : "Paris",
"PERCENT" : "vier Prozent",
"CARDINAL" : "drei"
}

In [ ]:
import pandas as pd
df = pd.read_csv("drive/MyDrive/Colab Notebooks/output_quest.csv")

In [ ]:
df.head(3)

In [ ]:
src_exm = df['src'][0]
src_exm

In [ ]:
edit_ner(src_exm, nlp, list_ent)

In [ ]:
mt_exm = df['mt'][0]
mt_exm

In [ ]:
edit_ner(mt_exm, nlp_ger, list_ent_german)

In [ ]:
doc = nlp(src_exm)
print(doc.ents)
for token in doc:
    print(token.text, token.pos_, token.dep_, token.ent_type_)

In [ ]:
doc = nlp_ger(mt_exm)
print(doc.ents)
for token in doc:
    print(token.text, token.pos_, token.dep_, token.ent_type_)

In [ ]:
df_tmp = df.head(5)
df_tmp

In [ ]:
df["new_src"] = df['src'].apply( lambda x : str(edit_ner((x), nlp, list_ent)) )

In [ ]:
df["new_mt"] = df['mt'].apply( lambda x : str(edit_ner((x), nlp_ger, list_ent_german)) )

In [ ]:
df.to_csv("drive/MyDrive/Colab Notebooks/output_quest_ner.csv")

In [ ]:
!pip install transquest

In [ ]:
import torch

from transquest.algo.sentence_level.monotransquest.run_model import MonoTransQuestModel


model = MonoTransQuestModel("xlmroberta", "TransQuest/monotransquest-da-multilingual", num_labels=1, use_cuda=torch.cuda.is_available())


In [ ]:
def transquest_model(src, tgt):    
    predictions, _ = model.predict([[src,tgt]])
    return round(predictions.item(),6)

In [ ]:
df = pd.read_csv("drive/MyDrive/Colab Notebooks/output_quest_ner.csv")

In [ ]:
df["tquest_ner"] = df.apply( lambda x : transquest_model(x["new_src"], x["new_mt"]) , axis=1)

In [ ]:
df.to_csv("drive/MyDrive/Colab Notebooks/output_tquest_ner.csv")

In [ ]:
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import accuracy_score

# Compute Pearson correlation coefficient
def pearsonr_metric (x, y):
    r, _ = pearsonr(y, x)
    return ("Pearson correlation coefficient: %.3f" % r)

# Compute mean absolute error
def mae_metric (x, y):
    mae = np.mean(np.abs(y - x))
    return ("Mean absolute error: %.3f" % mae)

# Compute root mean squared error
def rmse_metric (x, y):
    rmse = np.sqrt(np.mean((y - x)**2))
    return ("Root mean squared error: %.3f" % rmse)




In [ ]:
print("transquest results using spacy NER")
print(mae_metric(df["final_score"],df["tquest_ner"]))
print(rmse_metric(df["final_score"],df["tquest_ner"]))
print(pearsonr_metric(df["final_score"],df["tquest_ner"]))

In [ ]:
plot = df.plot.scatter(x="final_score", y="tquest_ner", title="tquest_ner")
plot